In [1]:
from ipywidgets import FloatProgress


In [2]:
import torch
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.device_count())
print("cuda current device:", torch.cuda.current_device())


torch version: 1.8.1+cu101
cuda available: 4
cuda current device: 0


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

device = 2
model_name = 'google/pegasus-xsum'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)


In [ ]:
src_text = [
    # """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow.""",
    """(CNN)Wanted: film director, must be eager to shoot footage of golden lassos and invisible jets. CNN confirms that Michelle MacLaren is leaving the upcoming "Wonder Woman" movie (The Hollywood Reporter first broke the story). MacLaren was announced as director of the movie in November. CNN obtained a statement from Warner Bros. Pictures that says, "Given creative differences, Warner Bros. and Michelle MacLaren have decided not to move forward with plans to develop and direct \'Wonder Woman\' together." (CNN and Warner Bros. Pictures are both owned by Time Warner.) The movie, starring Gal Gadot in the title role of the Amazon princess, is still set for release on June 23, 2017. It\'s the first theatrical movie centering around the most popular female superhero. Gadot will appear beforehand in "Batman v. Superman: Dawn of Justice," due out March 25, 2016. In the meantime, Warner will need to find someone new for the director\'s chair."""
]
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
batch

In [ ]:
class MyModel(nn.Module):
    def __init__(self, model_name):
        _model = PegasusForConditionalGeneration.from_pretrained(model_name)
        encoder = _model.model.encoder

In [6]:
model.model.encoder

PegasusEncoder(
  (embed_tokens): Embedding(96103, 1024, padding_idx=0)
  (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
  (layers): ModuleList(
    (0): PegasusEncoderLayer(
      (self_attn): PegasusAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
    (1): PegasusEncoderLayer(
      (self_attn): PegasusAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): 

In [8]:
additional_layer = torch.nn.Linear(1024, 1024)

In [13]:
my_model = torch.nn.ModuleList([model.model.encoder, additional_layer, model.model.decoder])

In [21]:
my_model.generate(**batch)

AttributeError: 'ModuleList' object has no attribute 'generate'

In [6]:
model.generate(**batch)

tensor([[    0,   198, 40328, 10339,   194,   148,   114,   177,  1758,   107,
             1]], device='cuda:2')

In [11]:
sample = [1060, 110, 107, 4139, 19219, 110, 107, 110, 82214, 4137, 151, 110, 107, 87454, 2000, 14956, 108, 377, 1408, 2191, 110, 107, 1426, 110, 107, 110, 58195, 151, 110, 107, 280, 96083, 14956, 108, 377, 1408, 2191, 110, 107, 202, 323, 113, 23992, 1843, 3083, 10078, 156, 113, 4329, 131, 116, 205, 12262, 16735, 112, 25873, 169, 1601, 131, 116, 1275, 117, 4319, 112, 109, 328, 131, 116, 6879, 34108, 238, 107, 2346, 7220, 108, 13042, 108, 140, 27301, 252, 173, 109, 11716, 111, 10738, 23992, 178, 4207, 195, 2515, 135, 2859, 19031, 144, 108, 40080, 108, 244, 169, 1601, 13580, 12326, 4812, 635, 204, 109, 850, 113, 109, 30361, 8348, 233, 162, 709, 10511, 5345, 113, 5328, 111, 114, 17159, 1669, 107, 1032, 109, 49877, 19328, 138, 129, 45207, 233, 115, 114, 14996, 156, 27960, 607, 153, 1319, 238, 107, 92928, 151, 139, 64054, 116, 113, 7220, 108, 13042, 108, 122, 156, 113, 109, 11716, 111, 10738, 23992, 2515, 135, 169, 30361, 34108, 238, 113, 2859, 19031, 144, 108, 40080, 107, 139, 3843, 10078, 342, 112, 25873, 169, 1601, 131, 116, 1275, 108, 109, 898, 109, 7543, 124, 1342, 110, 107, 41184, 151, 2346, 7220, 111, 169, 1601, 108, 1843, 1560, 117, 90117, 73215, 108, 115, 8985, 488, 134, 2859, 19031, 144, 12754, 981, 110, 107, 45876, 151, 614, 587, 135, 180, 1257, 606, 130, 109, 1034, 69249, 111, 10738, 14996, 131, 115, 109, 34108, 238, 110, 107, 13580, 12326, 4812, 108, 1843, 1560, 117, 90117, 73215, 108, 16849, 142, 6838, 328, 36177, 1678, 136, 232, 173, 169, 1802, 131, 116, 561, 121, 21425, 23992, 11623, 135, 2859, 19031, 144, 107, 139, 13639, 368, 146, 2265, 169, 1601, 131, 116, 1275, 134, 109, 68521, 34108, 238, 108, 3403, 109, 7543, 124, 1342, 151, 402, 187, 209, 374, 165, 559, 109, 3083, 196, 110, 107, 547, 107, 168, 123, 116, 161, 271, 123, 116, 201, 111, 178, 123, 116, 11242, 7370, 12323, 126, 107, 2346, 7220, 131, 116, 2045, 121, 386, 121, 5505, 9859, 5677, 43921, 10436, 108, 13073, 108, 13733, 109, 664, 3843, 140, 2145, 141, 114, 1707, 121, 768, 110, 107, 1034, 362, 123, 116, 3040, 161, 1289, 122, 342, 111, 125, 272, 123, 144, 393, 13309, 112, 626, 110, 107, 189, 820, 115, 169, 1275, 107, 123, 3054, 113, 4107, 169, 1601, 131, 116, 1275, 108, 2346, 7220, 111, 169, 1750, 5459, 445, 687, 112, 109, 1275, 113, 114, 110, 107, 328, 1151, 115, 10107, 107, 343, 169, 1601, 131, 116, 177, 1750, 140, 23107, 108, 1609, 126, 140, 114, 110, 19664, 111, 109, 23992, 195, 705, 115, 1112, 107, 9859, 5677, 43921, 10436, 108, 114, 1180, 6589, 170, 1257, 109, 211, 810, 23395, 25657, 21649, 173, 265, 2826, 13580, 12326, 4812, 136, 922, 108, 243, 151, 1034, 187, 137, 8323, 119, 120, 3546, 113, 109, 23992, 120, 195, 2255, 115, 295, 115, 150, 2760, 133, 174, 185, 11326, 12323, 1577, 1034, 159, 1304, 13580, 12326, 4812, 148, 1652, 133, 174, 221, 2679, 10303, 107, 131, 2859, 19031, 144, 117, 156, 113, 109, 3617, 2724, 113, 29327, 3105, 1]
len(sample)

512

In [17]:
tokenizer.vocab_size

96103

In [5]:
enc_out = model.model.encoder(**batch, output_hidden_states=True)

In [6]:
enc_out["last_hidden_state"].size()

torch.Size([2, 198, 1024])

In [7]:
enc_out["hidden_states"][0].size()

torch.Size([2, 198, 1024])

In [9]:
enc_out["last_hidden_state"][0][-1].size()

torch.Size([1024])

In [10]:
batch['input_ids']

tensor([[14887,   759,  1005,  3163,   126,  2798,   109, 25690,   116,   115,
          1407,   112, 13378,   118,   281,  7213, 10754,  1514,  1047,   107,
           139,  2560,   117,   112,  1329,   109,   887,   113, 39471,   107,
         16502,  6194,  4927,   527,   195,  2798,   112,   129,  2790,   141,
           109, 87338,   116,   162,   195,  1214,   112,   289,   224,   134,
           583, 26568,  3469,   107,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [6]:
eos_indices = (batch['input_ids'] == 1).nonzero(as_tuple=True)[-1]
eos_indices

tensor([ 54, 197], device='cuda:2')

In [7]:
a = batch['input_ids'][[i for i in range(eos_indices.size()[0])],eos_indices]
a

tensor([1, 1], device='cuda:2')

In [8]:
a = batch['input_ids'].index_select(1, eos_indices)
a

tensor([[  1,   0],
        [113,   1]], device='cuda:2')

In [13]:
eos_indices.size()[0]

2

In [20]:
sent_vecs = enc_out["last_hidden_state"][[i for i in range(eos_indices.size()[0])], eos_indices, :]
sent_vecs.size()

torch.Size([2, 1024])

In [9]:
enc_out["last_hidden_state"].size()

torch.Size([2, 198, 1024])

In [21]:
inner_products = torch.mv(enc_out["last_hidden_state"][0], sent_vecs[0])
inner_products.size()

torch.Size([198])

In [27]:
inner_products.median()

tensor(5.6273, device='cuda:2', grad_fn=<MedianBackward0>)

In [31]:
(inner_products > inner_products.median()).size()

torch.Size([198])

In [34]:
enc_out["last_hidden_state"][0, (inner_products > inner_products.median()), :].size()

torch.Size([99, 1024])

In [65]:
enc_out = model.model.encoder(**batch, output_hidden_states=True)
batch_size = 2
last_hidden_state = enc_out["last_hidden_state"]
last_hidden_state.retain_grad()
eos_indices = (batch['input_ids'] == 1).nonzero(as_tuple=True)[-1]
sent_vecs = last_hidden_state[[i for i in range(batch_size)], eos_indices, :]
inner_products = torch.mv(last_hidden_state[0], sent_vecs[0])
extracted_hidden_state = last_hidden_state[0, (inner_products > inner_products.median()), :]
extracted_hidden_state.mean().backward()
last_hidden_state.grad

tensor([[[9.8643e-06, 9.8643e-06, 9.8643e-06,  ..., 9.8643e-06,
          9.8643e-06, 9.8643e-06],
         [9.8643e-06, 9.8643e-06, 9.8643e-06,  ..., 9.8643e-06,
          9.8643e-06, 9.8643e-06],
         [9.8643e-06, 9.8643e-06, 9.8643e-06,  ..., 9.8643e-06,
          9.8643e-06, 9.8643e-06],
         ...,
         [9.8643e-06, 9.8643e-06, 9.8643e-06,  ..., 9.8643e-06,
          9.8643e-06, 9.8643e-06],
         [9.8643e-06, 9.8643e-06, 9.8643e-06,  ..., 9.8643e-06,
          9.8643e-06, 9.8643e-06],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.000

In [67]:
enc_out = model.model.encoder(**batch, output_hidden_states=True)
batch_size = 2
last_hidden_state = enc_out["last_hidden_state"]
last_hidden_state.retain_grad()
eos_indices = (batch['input_ids'] == 1).nonzero(as_tuple=True)[-1]
sent_vecs = last_hidden_state[[i for i in range(batch_size)], eos_indices, :]
sent_vecs = sent_vecs.unsqueeze(2)
inner_products = torch.matmul(last_hidden_state, sent_vecs)



# last_hidden_state.grad

In [42]:
sent_vecs = sent_vecs.unsqueeze(2)
inner_products = torch.matmul(last_hidden_state, sent_vecs)

In [43]:
inner_products.size()

torch.Size([2, 198, 1])

In [53]:
inner_products.reshape(batch_size, -1).median(axis=1)

torch.return_types.median(
values=tensor([6.7592, 2.6171], device='cuda:2', grad_fn=<MedianBackward1>),
indices=tensor([ 6, 30], device='cuda:2'))

In [75]:
inner_products = inner_products.reshape(batch_size, -1)
inner_products.median(dim=1)

torch.return_types.median(
values=tensor([5.6273, 5.6858], device='cuda:2', grad_fn=<MedianBackward1>),
indices=tensor([12, 28], device='cuda:2'))

In [73]:
torch.gt(inner_products, inner_products.median(axis=1))

TypeError: gt() received an invalid combination of arguments - got (Tensor, torch.return_types.median), but expected one of:
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, Number other, *, Tensor out)


In [78]:
inner_products.size()

torch.Size([2, 198])

In [89]:
inner_products.median(dim=1).values.unsqueeze(1).repeat(1, 198).size()

torch.Size([2, 198])

In [102]:
(inner_products > inner_products.median(dim=1).values.unsqueeze(1)).size()

torch.Size([2, 198])

In [103]:
last_hidden_state.size()

torch.Size([2, 198, 1024])

In [99]:
(inner_products > inner_products.median(dim=1).values.unsqueeze(1).repeat(1, 198))

tensor([[ True,  True,  True, False,  True, False,  True,  True, False, False,
          True, False, False, False,  True,  True, False,  True,  True,  True,
         False,  True, False, False,  True,  True,  True, False,  True,  True,
         False,  True,  True,  True, False,  True, False,  True,  True, False,
          True,  True,  True, False, False, False, False, False,  True, False,
         False,  True, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True, False,  True,  True, False,
         False, False,  True, False, False, False,  True, False,  True,  True,
          True,  True,  True,  True, False,  True,  True,  True,  True,  True,
          True,  True, False, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True, False, False, False,
         False, False, False, False, False, False, False, False,  True, False,
         False, False,  True,  True,  True,  True,  

In [98]:
last_hidden_state[, :].size()

torch.Size([198, 1024])